In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 50

In [ ]:
# Glossary of Comic Terms
# https://comics.ha.com/c/ref/glossary.zx

In [ ]:
data = pd.read_parquet('../data/all_data.parquet')

In [ ]:
data.info(memory_usage='deep')

In [ ]:
# get the proper date info
proper_date = pd.read_csv('../data/proper_date.csv')

In [ ]:
data['date'] = proper_date.proper_date.values

In [ ]:
titles = pd.read_csv('../data/titles_list_data.csv', low_memory=False, index_col=[0])

In [ ]:
titles.title.duplicated().value_counts()
# there are identical titles so we have to connect them with a unique id for convenience

In [ ]:
# extract titles' and issues' unique ids
data['title_id'] = data.copy().title_link.str.extract(pat=r'/series.(\d+)/\d.+')
data['issue_id'] = data.copy().issue_link.str.extract(pat=r'/series/\d+/(\d+)/.+')

In [ ]:
# keep relevant columns
df = data[['pub_name', 'title', 'title_id', 'issue', 'variant_of', 'volume', 'cover_date', 
           'years', 'date', 'cover_price', 'current_value', 'searched', 
          'owned', 'issues_total', 'est_print_run',  
           'pub_titles_total', 'pub_issues_total',
           'contributors_names', 'contributors_roles', 'characters', 'synopsis']]

In [ ]:
# turn prices into numerical
df.loc[:, 'cover_price'] = df.copy().loc[:, 'cover_price'].str.replace('[$,]', '', regex=True)
df.loc[:, 'cover_price'] = df.copy().loc[:, 'cover_price'].str.replace('Free', '0', regex=False)
df.loc[:, 'cover_price'] = df.copy().loc[:, 'cover_price'].apply(eval)

In [ ]:
df.loc[:, 'current_value'] = df.copy().loc[:, 'current_value'].str.replace('[$,]', '', regex=True)
df.loc[:, 'current_value'] = df.copy().loc[:, 'current_value'].str.replace('Free', '0', regex=False)
df.loc[:, 'current_value'] = df.copy().loc[:, 'current_value'].apply(eval)

In [ ]:
# Many issues with cover price 0 have high current value so we keep them
df[df.cover_price == 0].sort_values('current_value', ascending=False).head(2)

### `issue` column

There are several types of issues as indicated by their numbering and many unconventional numberings:

* Simple ascending numbering # 1-...

* Issues with numbering of the format: # (number)(variant-cover/ special ed./ convention ed. etc.): Normally these issues should be indicated as "variant of" (issue number) in the `variant_of` column

* 'Ashcan' Issues which are traditionally used to promote a new series (see [wiki](https://en.wikipedia.org/wiki/Ashcan_comic)). Ashcan comics can be quite rare and valuable, especially from the [Golden Age](https://en.wikipedia.org/wiki/Golden_Age_of_Comic_Books) (30s-50s)

* Issues with `nn` meaning 'non-numbered'. Typically 0th issues or one-shots

* Of "Vol ** # **" format (where ** indicated a number). i.e. two enumerations are indicated, one for volume, one for issue number

* Instead of issue number a date is given e.g. "Spring 2007" or simply "1989"

* Numbering which includes letters e.g. "C-2" or "x"

* Just the name of the protagonist or in general something explanatory e.g. "Batwoman"

* The print run is indicated e.g. '# 2 - 2nd print'


In [ ]:
df1 = df.copy()

In [ ]:
# get the print number
prints = df1.issue.str.extract('.*(\d{1,2})[a-zA-Z]{2}[\s\-][pP]rint')
df1['print_nr'] = prints.fillna(1).astype(int)
# extract volume and issue information from issue of the type e.g. 'V5-14' or 'V9 6' or 'V12#6'
vols = (df1.issue.str.extractall('.*[Vv][oO]?[lL]?(\d+)[\s\-#]+(\d+)')
            .droplevel(1).rename({0: 'vol', 1: 'issue'}, axis=1))
# extract volume and issue information from issue of the type e.g. 'B-14' or 'C 6'
lett = df1.issue.str.extractall('([A-Z])[#\s\-](\d+)\s?.*').droplevel(1).rename({0: 'vol', 1: 'issue'}, axis=1)

In [ ]:
# Find all ashcan issues
idx_ash = df1.issue.str.contains('[Aa]shcan')
# Find all non-numbered and ashcan issues
idx_nn = df1.issue.str.contains('(?:\s+|^)nn-?(?:\s*|$)')
df1.loc[idx_nn, 'volume'] = 'one-shot'
# Find all previews
idx_prev = df1.issue.str.contains('[Pp]review')
# find all issues that contain also the volume information
vols_idx = df1.issue.str.contains('.*[Vv][oO]?[lL]?#?\d+[\s\-#]+\d+')
# find issues that are properly numbered
numb_idx = df1.issue.str.contains('.*#\s*-?\d+\D*.*')
# find issues that are numbered as e.g. B-5 etc.
lett_idx = df1.issue.str.contains('^[A-Z][#\s\-]\d+\s?.*')
# find issues that are numbered by single capital letters
caplet_idx = df1.issue.str.contains('^[A-Z]$')
# find issues that are numbered by single lowercase letters
lowlet_idx = df1.issue.str.contains('^[a-z]$')
# find issues from free comic book day
fcbd_idx = df1.issue.str.contains('fcbd|free comic')
df1.loc[fcbd_idx, 'volume'] = 'fcbd'

# Extract issue numbers by the ones that are numbered in the conventional way
numbered = df1.issue.str.extract(r'#\s*(\d+)\D*')

# ashcan --> -1
numbered.loc[idx_ash] = str(0)
# nn --> 1
numbered.loc[idx_nn] = str(1)
# preview --> 0.5
numbered.loc[idx_prev] = str(0.5)
# free comic book day (fcbd) --> 1
numbered.loc[fcbd_idx] = str(1)
# single capital letter numbering to int numbers
numbered.loc[caplet_idx] = df1[caplet_idx].issue.apply(lambda x: str(ord(x) - 64)).values.reshape(-1, 1)
# single lower letter numbering to int numbers
numbered.loc[lowlet_idx] = df1[lowlet_idx].issue.apply(lambda x: str(ord(x) - 96)).values.reshape(-1, 1)

# add volume and issue info extracted from issue column
numbered.loc[vols_idx, 0] = vols.issue 
df1.loc[vols_idx, 'volume'] = vols.vol
numbered.loc[lett_idx, 0] = lett.issue
df1.loc[lett_idx, 'volume'] = lett.vol.apply(lambda x: str(ord(x) - 64))

# create new column with the issues numbers
df1.insert(4, 'issue_nr', pd.to_numeric(numbered[0]))

In [ ]:
# find all the above
regex = r'(?:\s+|^)nn-?(?:\s*|$)|.*#\s*-?\d+\D*.*|[Aa]shcan|[Pp]review|.*[Vv]\d+\s?-?\s?\d+|[A-Z][\s-]\d+\s?.*|^[A-Z]$|fcbd|free comic|^[a-z]$'
idx_good = (df1.issue.str.contains(regex))

In [ ]:
# All that remained that we will drop since issue number is considered essential
print(df1[~idx_good].shape)
print(df1[~idx_good].current_value.sum())

In [ ]:
df2 = df1.dropna(subset=['issue_nr'])
# drop issues with issue_nr > 10000 - considered outliers
df2 = df2[~(df2.issue_nr > 10000)]

### `volume` column

In [ ]:
# All none values for volume can be considered volume 1
df2.volume.isna().sum()

In [ ]:
# take care of volume numbers formatted as e.g. 'Vol. 5'
vol_nr = df2.volume.str.extract('.*[vV][oO][lL][.\s\-]+(\d+)')
vol_nr_idx = vol_nr.notna()[0]

df2.loc[vol_nr_idx, 'volume'] =  vol_nr

In [ ]:
df2.volume.fillna('0', inplace=True)

In [ ]:
# Volume of type 'one-shot'
oneshot_idx = df2.volume.str.contains('one[\s\-=]+shot', flags=re.IGNORECASE)
df2.loc[oneshot_idx, 'volume'] = 'one-shot'
# Volume of type trade paper-back (TPB) or soft-cover (SC)
tpb_idx = df2.volume.str.contains('TRPB|TPB|TP|paperback|sc|soft', flags=re.IGNORECASE)
df2.loc[tpb_idx, 'volume'] = 'TPB/SC'
# Volume of type Hard Cover (HC)
hc_idx = df2.volume.str.contains('HC|hard', flags=re.IGNORECASE)
df2.loc[hc_idx, 'volume'] = 'HC'
# Volume of type graphic novel (GN)
gn_idx = df2.volume.str.contains('^gn$|^ogn$', flags=re.IGNORECASE)
df2.loc[gn_idx, 'volume'] = 'GN'
# Volume of type mini-series/ maxi-series /limited-series which are ofter used interchangeably (https://en.wikipedia.org/wiki/Limited_series_(comics))
mini_idx = df2.volume.str.contains('mini|maxi|limit', flags=re.IGNORECASE)
df2.loc[mini_idx, 'volume'] = 'limited-series'
# Volume of type magazine
mag_idx = df2.volume.str.contains('magazine', flags=re.IGNORECASE)
df2.loc[mag_idx, 'volume'] = 'magazine'
# Volume of type fanzine
fan_idx = df2.volume.str.contains('fanzine', flags=re.IGNORECASE)
df2.loc[fan_idx, 'volume'] = 'fanzine'
# Volume for fcbd comics
FCBD_idx = df2.volume.str.contains('fcbd', flags=re.IGNORECASE)
df2.loc[FCBD_idx, 'volume'] = 'fcbd'
# Volume for four-colour comics
four_idx = df2.volume.str.contains('(?=.*[fF]our)(?=.*[cC]olor|.*[cC]olour)', flags=re.IGNORECASE)
df2.loc[four_idx, 'volume'] = 'four-color'

In [ ]:
volume_cats = ['limited-series', 'one-shot', 'magazine', 'TPB/SC', 'HC', 'GN', 'four-color', 
               'Prepack', 'fcbd', 'fanzine']
volume_nums = list(range(21))
volume_lst = volume_cats + [str(n) for n in volume_nums]

In [ ]:
other_idx = ~df2.volume.isin(volume_lst)

In [ ]:
# get rid of those, in most cases these are not comic books
df3 = df2[~other_idx]

### `variant_of` column

In [ ]:
# use info in variant to fill volume information
variant = df3[df3.variant_of.notna()]

In [ ]:
# find variant number in the form 'Soft Cover or TPB # 213'
tpb_var_idx = variant.variant_of.str.contains('TRPB|TPB|TP|paperback|sc|soft', flags=re.IGNORECASE)
# extract issue number
tpb_var_num = variant[tpb_var_idx].variant_of.str.extract(r'.*#?\s*(\d+)\D*')

# find variant number in the form 'Hard Cover # 213'
hc_var_idx = variant.variant_of.str.contains('hard|HC', flags=re.IGNORECASE)
# extract issue number
hc_var_num = variant[hc_var_idx].variant_of.str.extract(r'.*#?\s*(\d+)\D*')

In [ ]:
var_tpb_index = tpb_var_idx[(tpb_var_idx == True)].index
df3.loc[var_tpb_index, 'volume'] = 'TPB/SC'

var_hc_index = tpb_var_idx[(hc_var_idx == True)].index
df3.loc[var_hc_index, 'volume'] = 'HC'

In [ ]:
# Make new column where variant is bool True or False
is_variant = df3.variant_of.notna()

In [ ]:
df3.loc[:, 'variant'] = is_variant.copy()

### `synopsis` column

Try to find other information from the synopsis of each issue e.g. if it is an incentive cover variant

In [ ]:
# pd.options.display.max_colwidth = 1000
# pd.options.display.max_rows = 500

In [ ]:
df3.synopsis.notna().value_counts()

In [ ]:
df3[df3.synopsis.notna()].synopsis.sample(1)

In [ ]:
incentives = df3.synopsis.str.contains('incentive').fillna(False)

In [ ]:
limited_idx = df3.synopsis.str.contains('(?=.*limited)(?=.*edition|.*cover|.*variant|.*issue|.*copies|.*copy)', flags=re.IGNORECASE).fillna(False)

In [ ]:
limit_copies = df3[limited_idx].synopsis.str.extract(r'.*\s(\d,?\d+) copies.*|.*limited to (\d,?\d+).*|.*1:(\d,?\d+).*', flags=re.IGNORECASE)

In [ ]:
limit_copies = limit_copies[0].fillna(limit_copies[1].fillna(limit_copies[2]))

In [ ]:
# deluxe edition (< limited)
deluxe_idx = df3.synopsis.str.contains('(?=.*deluxe)(?=.*edition)', flags=re.IGNORECASE).fillna(False)

In [ ]:
# create a column 'special' to indicate limited/deluxe edition etc
df3.loc[:, 'special'] = 'not'

In [ ]:
df3.loc[deluxe_idx, 'special'] = 'deluxe'
df3.loc[limited_idx, 'special'] = 'limited'

In [ ]:
df3.special.value_counts()

In [ ]:
# The number of copies for the limited editions wherever it could be obtained
limit_copies = limit_copies.str.replace(',', '').astype(float).dropna()

#### Get first appearance and maybe other special events from `characters` column 

In [ ]:
def get_events(char_list):
    
    if char_list == 'no characters assigned':
        return 'nothing'
    else:
        chars = eval(char_list)
        events = []
        for char in chars:
            if char[-1] != '':
                events.append(char[-1])
            else:
                continue
        if events == []:
            return 'nothing'
        else:
            return str(events)
            

In [ ]:
lst = df3.characters[0]

In [ ]:
lst

In [ ]:
events = df3.characters.apply(get_event)

In [ ]:
df3.loc[:, 'first_appear_event'] = events.str.contains('First appearance', flags=re.IGNORECASE)
df3.loc[:, 'death_event'] = events.str.contains('Death', flags=re.IGNORECASE)
df3.loc[:, 'origin_event'] = events.str.contains('origin', flags=re.IGNORECASE)

### Prepare final dataset

In [ ]:
df4 = df3.copy()

In [ ]:
df4.info()

In [ ]:
df4 = df4[['pub_name', 'title', 'title_id', 'issue_nr', 'variant', 'volume', 'print_nr', 
          'date', 'cover_price', 'current_value', 'special', 'searched', 'owned',
         'first_appear_event', 'death_event', 'origin_event', 'issues_total',
         'pub_titles_total', 'pub_issues_total']]

In [ ]:
df4.info(memory_usage='deep')

In [ ]:
df4.to_csv('../data/data_clean.csv')